# Blurred data with Poisson noise

### Importing libraries, generating data and operators

In [ ]:
#Importing libraries
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import scipy
import scipy.io
from PIL import Image
import time
import holoviews as hv
import panel as pn
from scipy import signal
hv.extension('bokeh')

In [ ]:
#Importing libraries
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import scipy.linalg as spl
import time

In [ ]:
#Importing additional packages: algorithms.py contains the algorithms to apply and visualizer.py is useful to plot loss functions.
import os.path
if not os.path.isfile('./visualizer.py'):
    !wget https://github.com/HippolyteLBRRR/Composite_optimization/raw/main/visualizer.py
if not os.path.isfile('./algorithms.py'):
    !wget https://github.com/HippolyteLBRRR/Composite_optimization/raw/main/algorithms.py
from algorithms import *
from visualizer import *

In [ ]:
#Réglage pour l'affichage des images
options0=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None,cmap='hot')

In [ ]:
#Import des données
pth_kernel='./ISBI_1.mat'
data = scipy.io.loadmat(pth_kernel)
print(data)
im=data["gt"]
kern=data["PSF"]
M=data["M"]
bg=data["b"]
im_det=data["Acq"]+bg
pn.Row(hv.Image(im).opts(**options0),hv.Image(im_det).opts(**options0),hv.Image(kern).opts(**options0))

In [ ]:
xmin = FISTA_automatic_restart(xAR1,1/L,5000,0,Df,proxh,F,exit_crit=exit_crit,out_cost=False)

In [ ]:
#Définition des opérateurs de convolution et d'un bruit de Poisson

def Convolution(x, A):
    #Attention, A n'est pas le noyau mais sa fft.
    y = np.real(np.fft.ifft2(A * np.fft.fft2(x)))
    return y
    #return signal.convolve2d(x,h,mode='same',boundary='fill',fillvalue=1)

def Convolution_adj(x,A):
    #Attention, A n'est pas le noyau mais sa fft.
    y = np.real(np.fft.ifft2(np.conj(A) * np.fft.fft2(x)))
    return y

def SousEchantillonage(x,M):
    return M@x@M.T

def Poisson(x):
    return np.random.poisson(x)

def KL1(H,M,x,z,bg):
    MHx=SousEchantillonage(H(x),M)
    temp=(z!=0)*(z*np.log(z/(MHx+bg)))+MHx+bg-z
    return np.sum(temp)
def Grad_KL1(H,HT,M,x,z,bg):
    MHx=SousEchantillonage(H(x),M)
    return HT(SousEchantillonage(np.ones(MHx.shape)-z/(MHx+bg),M.T))

def operators_poisson(kern,M,lmbd,im,bg):
    #kern : noyau de convolution
    #lmbd : coefficient de régularisation
    #im : image détériorée initiale
    #bg : valeur du background
    N=np.max(np.shape(M))
    n=np.min(np.shape(M))
    PSF = np.fft.fft2(np.fft.fftshift(kern), [N,N])
    Convo = lambda x: Convolution(x,PSF)
    ConvoT = lambda x: Convolution_adj(x,PSF)
    f = lambda x: KL1(Convo,M,x,im,bg)
    Df = lambda x: Grad_KL1(Convo,ConvoT,M,x,im,bg)     
    h = lambda x: lmbd*np.sum(np.abs(x))
    proxh = lambda x,s: (x-lmbd*s)*(x>lmbd*s)
    F = lambda x: f(x)+h(x)
    im_init=Convo(SousEchantillonage(im,M.T))
    return F,f,h,Df,proxh,im_init

In [ ]:
#Récupération des opérateurs f,h,gradf,proxh et définition du critère de sortie et du produit scalaire de référence
lmbd=8
F,f,h,Df,proxh,im_init=operators_poisson(kern,M,lmbd,im_det,bg)
exit_crit = lambda x,y: npl.norm(np.ravel(x-y),2)
sp = lambda x,y: np.dot(np.ravel(x),np.ravel(y))
hv.Image(im_init).opts(**options0)

In [ ]:
#Paramètres pour la résolution du pb (Niter:nombre max d'ité, L0:estimation de L choisie arbitrairement, epsilon:précision souhaitée)
Niter=1000
L0=100
epsilon=1e-6

In [ ]:
rho=0.8
delta=0.95
xBT,cost_BT,ctime_BT,Lk=FISTA_BT(im_init, L0, rho, delta, Niter, epsilon, f, Df, proxh, h, exit_crit, sp,out_L=True,exit_norm=True,track_ctime=True)

In [ ]:
Plot_BT=To_Plot(cost_BT,"FISTA with backtracking, rho=%f, delta=%f, L0=%i"%(rho,delta,L0),ctime_BT)

In [ ]:
rho=0.8
delta=0.95
xRBT,cost_BTR,ctime_BTR,tab_L,tab_mu,irestartBTR=Free_FISTA(im_init, L0, rho, delta, Niter, epsilon, f, h, Df, proxh,exit_crit=exit_crit, sp=sp, out_cost=True,out_L=True,exit_norm=True,out_condition=True,track_ctime=True,track_restart=True)

In [ ]:
Plot_BTR=To_Plot(cost_BTR,"Free-FISTA, rho=%f, delta=%f, L0=%i"%(rho,delta,L0),ctime_BTR,extra_points=irestartBTR)

In [ ]:
Plot([Plot_BT,Plot_BTR],ite=False,eps=1e-5,fontsize = 25)
plt.xlabel("Computation time (in seconds)",fontsize=25)
plt.ylabel("$\log(F(x_k)-\hat F)$",fontsize=25)

In [ ]:
hv.Image(xBT).opts(**options0)